[refrence link on how to build rag with chat history](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/#chains)

In [1]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredHTMLLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from typing import List
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv, find_dotenv
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from fastapi import HTTPException

import os
import logging

import sys
sys.path.append('/home/phucuy2025/RAG-Chatbot/api')
sys.path.append('/home/phucuy2025/RAG-Chatbot')
from api.scraper import WebScraper


gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)
#$ initialize the MongoDB python client
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(
    MONGODB_ATLAS_CLUSTER_URI
)
DB_NAME = "RAG-Chatbot-Cluster"
COLLECTION_NAME = "RAG-Chatbot-Collection-Test"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "RAG-Chatbot-Index-Test"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=gemini_embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine"
)

def initialize_vector_store():
    """Initialize the MongoDB collection and verify the vector search index."""
    try:
        # Verify MongoDB connection
        client.server_info()  # Raises an exception if connection fails
        logging.info("MongoDB connection established successfully")

        # Check if collection exists
        if COLLECTION_NAME not in client[DB_NAME].list_collection_names():
            client[DB_NAME].create_collection(COLLECTION_NAME)
            logging.info(f"Created collection {COLLECTION_NAME}")
        else:
            logging.info(f"Collection {COLLECTION_NAME} already exists")

        # Note: Vector search index must be created in MongoDB Atlas UI or via API
        logging.info(f"Ensure vector search index '{ATLAS_VECTOR_SEARCH_INDEX_NAME}' is configured in MongoDB Atlas for collection {COLLECTION_NAME}")

        # Test vector store by adding a dummy document
        dummy_doc = Document(page_content="Test document", metadata={"file_id": 0})
        vector_store.add_documents([dummy_doc])
        logging.info("Added test document to vector store")

        # Log the inserted document to inspect its structure
        inserted_doc = vector_store._collection.find_one({"file_id": 0})
        if inserted_doc:
            logging.info(f"Inserted test document: {inserted_doc.get('file_id')}")
        else:
            logging.error("Test document not found after insertion")

        # Delete the test document
        result = vector_store._collection.delete_one({"file_id": 0})
        if result.deleted_count > 0:
            logging.info("Successfully deleted test document")
        else:
            logging.warning("No test document was deleted; check document structure or query")

        # Verify deletion
        remaining_doc = vector_store._collection.find_one({"file_id": 0})
        if remaining_doc:
            logging.error(f"Test document still exists after deletion attempt: {remaining_doc}")
        else:
            logging.info("Confirmed test document was deleted")

    except Exception as e:
        logging.error(f"Failed to initialize vector store: {str(e)}")
        raise
def delete_collection():
    """Delete the entire MongoDB collection."""
    try:
        client[DB_NAME].drop_collection(COLLECTION_NAME)
        print(f"Successfully deleted collection {COLLECTION_NAME}")
        return True
    except Exception as e:
        print(f"Error deleting collection {COLLECTION_NAME}: {str(e)}")
        return False



In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import List
from langchain_core.documents import Document
import os
import logging
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

# from api.mongo_db_utils import vector_store
# retriever = vector_store.as_retriever(search_kwargs={"k": 2})
retriever = vector_store.as_retriever()
output_parser = StrOutputParser()

logging.basicConfig(filename="rag_chatbot_app.log", level=logging.INFO)


# Set up prompts and chains

contextualize_q_system_prompt = (
    "Với lịch sử trò chuyện và câu hỏi mới nhất của người dùng, "
    "câu hỏi có thể tham chiếu đến ngữ cảnh trong lịch sử trò chuyện, "
    "hãy xây dựng một câu hỏi độc lập có thể hiểu được"
    "mà không cần lịch sử trò chuyện. KHÔNG trả lời câu hỏi, "
    "chỉ cần xây dựng lại câu hỏi nếu cần và nếu không thì giữ nguyên câu hỏi như hiện tại."
)

contextualize_q_prompt = ChatPromptTemplate([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

system_prompt = (
    "Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi. "
    "Sử dụng các phần ngữ cảnh sau đây để trả lời "
    "câu hỏi. Nếu bạn không biết câu trả lời, hãy nói rằng bạn "
    "không biết. Sử dụng tối đa ba câu và giữ cho câu trả lời ngắn gọn."
     "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)



def get_rag_chain(model="gemini-2.0-flash-001"):
    llm = ChatGoogleGenerativeAI(model=model, google_api_key=GOOGLE_API_KEY)
    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    # Modify the question_answer_chain to return both answer and source documents
    
    
    return rag_chain

In [4]:
from api.scraper import WebScraper

def load_and_split_html(base_url: str) -> List[Document]:
    # base_url = "https://apidog.com/vi/blog/rag-deepseek-r1-ollama-vi/"  # Replace with your target website
    scraper = WebScraper(base_url)
    # print("step 1")
    # Scrape the website
    scraped_content = scraper.scrape_website(max_pages=1)
    # print("step 2")
    # Process and split the content
    processed_chunks = scraper.process_content(scraped_content)
    # print("step 3")
    documents = []
    for chunk in processed_chunks:
        documents.append(
            Document(
                page_content = chunk.get('content'),
                metadata = {"source": chunk.get('url')}
            )
        )

    # Print results
    print(f"Total pages scraped: {len(scraped_content)}")
    print(f"Total chunks created: {len(processed_chunks)}")
    
    # Example of accessing the first chunk
    return documents

documents = load_and_split_html(base_url='https://vnexpress.net/cam-nang-du-lich-mot-ngay-kham-pha-dia-dao-cu-chi-4873164.html')
documents

Total pages scraped: 1
Total chunks created: 12


[Document(metadata={'source': 'https://vnexpress.net/cam-nang-du-lich-mot-ngay-kham-pha-dia-dao-cu-chi-4873164.html'}, page_content='Cẩm nang du lịch một ngày khám phá địa đạo Củ Chi Du lịchĐiểm đến Thứ hai, 1442025, 0700 GMT7 Một ngày khám phá địa đạo Củ Chi TP HCMChui hầm địa đạo, thăm khu vực tái hiện vùng giải phóng, thưởng thức món ăn dân dã là những trải nghiệm giúp du khách hiểu hơn về cuộc sống thời chiến của quân dân vùng đất thép. Cách trung tâm TP HCM khoảng 70 km về hướng Tây Bắc, địa đạo Củ Chi với hệ thống đường hầm dài gần 250 km, là cứ địa vững chắc của Khu ủy Quân khu, Bộ tư lệnh Sài Gòn - Gia Định, góp phần không nhỏ vào công cuộc thống nhất đất nước. Hiện nay, di tích địa đạo Củ Chi được bảo tồn tại hai khu vực chính là Bến Dược, xã Phú Mỹ Hưng và Bến Đình, xã Nhuận Đức, trở thành điểm đến thu hút du khách khi đến TP HCM. Theo đại diện Khu di tích lịch sử địa đạo Củ Chi, lượng khách tham quan trong tháng 4 tăng 30 so với ngày thường, nhờ hiệu ứng từ chuỗi sự kiện kỷ 

In [5]:
initialize_vector_store()
for split in documents:
            # add file id to the metadata of each split
    split.metadata['file_id'] = 0
        
        # add the document chunks to the vector store
vector_store.add_documents(documents)

documents = load_and_split_html(base_url='https://lilianweng.github.io/posts/2023-06-23-agent/')
for split in documents:
            # add file id to the metadata of each split
    split.metadata['file_id'] = 1
        
        # add the document chunks to the vector store
vector_store.add_documents(documents)

Total pages scraped: 1
Total chunks created: 52


['68062d03e1c9a1c2050c6834',
 '68062d03e1c9a1c2050c6835',
 '68062d03e1c9a1c2050c6836',
 '68062d03e1c9a1c2050c6837',
 '68062d03e1c9a1c2050c6838',
 '68062d03e1c9a1c2050c6839',
 '68062d03e1c9a1c2050c683a',
 '68062d03e1c9a1c2050c683b',
 '68062d03e1c9a1c2050c683c',
 '68062d03e1c9a1c2050c683d',
 '68062d03e1c9a1c2050c683e',
 '68062d03e1c9a1c2050c683f',
 '68062d03e1c9a1c2050c6840',
 '68062d03e1c9a1c2050c6841',
 '68062d03e1c9a1c2050c6842',
 '68062d03e1c9a1c2050c6843',
 '68062d03e1c9a1c2050c6844',
 '68062d03e1c9a1c2050c6845',
 '68062d03e1c9a1c2050c6846',
 '68062d03e1c9a1c2050c6847',
 '68062d03e1c9a1c2050c6848',
 '68062d03e1c9a1c2050c6849',
 '68062d03e1c9a1c2050c684a',
 '68062d03e1c9a1c2050c684b',
 '68062d03e1c9a1c2050c684c',
 '68062d03e1c9a1c2050c684d',
 '68062d03e1c9a1c2050c684e',
 '68062d03e1c9a1c2050c684f',
 '68062d03e1c9a1c2050c6850',
 '68062d03e1c9a1c2050c6851',
 '68062d03e1c9a1c2050c6852',
 '68062d03e1c9a1c2050c6853',
 '68062d03e1c9a1c2050c6854',
 '68062d03e1c9a1c2050c6855',
 '68062d03e1c9

In [14]:
from pprint import pprint
pprint(MONGODB_COLLECTION.find_one().keys())
print(vector_store._collection.count_documents({}))

# create vector search index
from pymongo.mongo_client import MongoClient
from pymongo.operations import SearchIndexModel

def create_index():
    # Connect to your Atlas deployment
    client = MongoClient(
        MONGODB_ATLAS_CLUSTER_URI
    )
    DB_NAME = "RAG-Chatbot-Cluster"
    COLLECTION_NAME = "RAG-Chatbot-Collection-Test"
    ATLAS_VECTOR_SEARCH_INDEX_NAME = "RAG-Chatbot-Index-Test"

    collection = client[DB_NAME][COLLECTION_NAME]

    # Create your index model, then create the search index
    search_index_model = SearchIndexModel(
                definition={
                    "mappings": {
                        "dynamic": True,
                        "fields": {
                            "embedding": {  # Correct structure: field name as key
                                "type": "knnVector",
                                "dimensions": 768,
                                "similarity": "cosine"
                            }
                        }
                    }
                },
                name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
            )
    
    result = collection.create_search_index(model=search_index_model)
    print(result)

create_index()


dict_keys(['_id', 'text', 'embedding', 'source', 'file_id'])
64
RAG-Chatbot-Index-Test


In [15]:
query = "Bạn có biết địa đạo Củ Chi không? Hãy cho tôi một vài thông tin về địa điểm này."
compressed_docs = vector_store.similarity_search(query, k=2)
compressed_docs

[Document(id='68062d00e1c9a1c2050c6832', metadata={'_id': '68062d00e1c9a1c2050c6832', 'source': 'https://vnexpress.net/cam-nang-du-lich-mot-ngay-kham-pha-dia-dao-cu-chi-4873164.html', 'file_id': 0}, page_content='còn có thể trải nghiệm nhiều hoạt động giải trí ngoài trời như bắn súng thể thao, câu cá thư giãn hoặc tham gia các trò chơi dân gian tại khu vực sân vườn. Một số điểm trong khuôn viên còn tổ chức biểu diễn tái hiện cảnh chiến đấu, giúp du khách hiểu rõ hơn về cuộc sống và tinh thần thép của quân dân Củ Chi trong kháng chiến. Hàng quán trong chợ được kê trên bàn tre thấp, khách ngồi trên ghế chọn món rồi thưởng thức tại chỗ hoặc mang thức ăn vào các mái nhà lá cạnh bên có bố trí ghế ngồi và chỗ thoáng mát. Chợ còn bán các món ăn Nam bộ như bánh xèo, cá lóc nướng trui, gỏi cuốn. Bên cạnh hành trình khám phá địa đạo, du khách đến Củ Chi còn có thể trải nghiệm nhiều hoạt động giải trí ngoài trời như bắn súng thể thao, câu cá thư giãn hoặc tham gia các trò chơi dân gian tại khu vự

Query directly vector store

In [40]:
assert ATLAS_VECTOR_SEARCH_INDEX_NAME == 'RAG-Chatbot-Index-Test'

In [45]:
vector_store2._collection.count_documents({})

64

In [47]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_google_genai import ChatGoogleGenerativeAI

from pymongo import MongoClient
# reconnect

DB_NAME = "RAG-Chatbot-Cluster"
COLLECTION_NAME = "RAG-Chatbot-Collection-Test"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "RAG-Chatbot-Index-Test"
# MONGODB_ATLAS_CLUSTER_URI
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
db = client[DB_NAME]
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store2 = MongoDBAtlasVectorSearch(
    MONGODB_COLLECTION,
    embedding=gemini_embeddings,
    index_name='RAG-Chatbot-Index-Test'

)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_store.as_retriever()
)

query = "What is task decomposition?"
query = "Bạn có biết địa đạo Củ Chi không? Hãy cho tôi một vài thông tin về địa điểm này."
query = "Hệ thống đường hầm của nó dài bao nhiêu?"
compressed_docs = vector_store.similarity_search(query, k=2)
compressed_docs

[Document(id='680628a6d2a57cd5943c9d71', metadata={'_id': '680628a6d2a57cd5943c9d71', 'source': 'https://vnexpress.net/cam-nang-du-lich-mot-ngay-kham-pha-dia-dao-cu-chi-4873164.html', 'file_id': 0}, page_content='năm thống nhất đất nước. Đơn vị dự báo lượng khách sẽ tiếp tục tăng cao trong dịp lễ 304 sắp tới. Trong ảnh là sơ đồ lát cắt bên trong hệ thống đường hầm của địa đạo. Sau khi viếng Đền Bến Dược, du khách di chuyển bằng xe điện khoảng một km để đến khu vực địa đạo. Một số đoạn hầm đã được cải tạo, mở rộng lối vào tạo thuận tiện cho khác tham quan. Hệ thống địa đạo bao gồm các đường dẫn đến nhiều khu chức năng như hầm ở và làm việc của lãnh đạo, hầm y tế, khu vực ăn uống, kho chứa lương thực và vũ khí, ô chiến đấu, giếng nước, bếp Hoàng Cầm, công binh xưởng và nhà may quân trang. Tại mỗi khu vực, hướng dẫn viên sẽ dẫn đoàn đi tham quan và thuyết minh chi tiết về các công trình, di tích cũng như những điểm tái hiện lịch sử. Sau khi viếng Đền Bến Dược, du khách di chuyển bằng xe đ

In [38]:
res = vector_store2._collection.find({"file_id": 0})# 6805fdc9101024e2fd3a72e4
res.to_list()
    # print(i)

[{'_id': ObjectId('680628a6d2a57cd5943c9d6f'),
  'text': 'Cẩm nang du lịch một ngày khám phá địa đạo Củ Chi Du lịchĐiểm đến Thứ hai, 1442025, 0700 GMT7 Một ngày khám phá địa đạo Củ Chi TP HCMChui hầm địa đạo, thăm khu vực tái hiện vùng giải phóng, thưởng thức món ăn dân dã là những trải nghiệm giúp du khách hiểu hơn về cuộc sống thời chiến của quân dân vùng đất thép. Cách trung tâm TP HCM khoảng 70 km về hướng Tây Bắc, địa đạo Củ Chi với hệ thống đường hầm dài gần 250 km, là cứ địa vững chắc của Khu ủy Quân khu, Bộ tư lệnh Sài Gòn - Gia Định, góp phần không nhỏ vào công cuộc thống nhất đất nước. Hiện nay, di tích địa đạo Củ Chi được bảo tồn tại hai khu vực chính là Bến Dược, xã Phú Mỹ Hưng và Bến Đình, xã Nhuận Đức, trở thành điểm đến thu hút du khách khi đến TP HCM. Theo đại diện Khu di tích lịch sử địa đạo Củ Chi, lượng khách tham quan trong tháng 4 tăng 30 so với ngày thường, nhờ hiệu ứng từ chuỗi sự kiện kỷ niệm 50 năm thống nhất đất nước. Đơn vị dự báo lượng khách sẽ tiếp tục tăng

In [48]:
vector_store2 = MongoDBAtlasVectorSearch(
    MONGODB_COLLECTION,
    embedding=gemini_embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
)
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 2, "score_threshold": 0.5},  # Allow all scores
)
query = "Hệ thống đường hầm của nó dài bao nhiêu?"
results = retriever.invoke(query, k=3)
results

[Document(id='680628a6d2a57cd5943c9d71', metadata={'_id': '680628a6d2a57cd5943c9d71', 'source': 'https://vnexpress.net/cam-nang-du-lich-mot-ngay-kham-pha-dia-dao-cu-chi-4873164.html', 'file_id': 0}, page_content='năm thống nhất đất nước. Đơn vị dự báo lượng khách sẽ tiếp tục tăng cao trong dịp lễ 304 sắp tới. Trong ảnh là sơ đồ lát cắt bên trong hệ thống đường hầm của địa đạo. Sau khi viếng Đền Bến Dược, du khách di chuyển bằng xe điện khoảng một km để đến khu vực địa đạo. Một số đoạn hầm đã được cải tạo, mở rộng lối vào tạo thuận tiện cho khác tham quan. Hệ thống địa đạo bao gồm các đường dẫn đến nhiều khu chức năng như hầm ở và làm việc của lãnh đạo, hầm y tế, khu vực ăn uống, kho chứa lương thực và vũ khí, ô chiến đấu, giếng nước, bếp Hoàng Cầm, công binh xưởng và nhà may quân trang. Tại mỗi khu vực, hướng dẫn viên sẽ dẫn đoàn đi tham quan và thuyết minh chi tiết về các công trình, di tích cũng như những điểm tái hiện lịch sử. Sau khi viếng Đền Bến Dược, du khách di chuyển bằng xe đ

In [64]:
results = vector_store.similarity_search(
    "Bạn có biết địa đạo Củ Chi không? Hãy cho tôi một vài thông tin về địa điểm này.",
    k=2
)
print(f"results = {results}")
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

results = []


In [9]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
rag_chain = get_rag_chain()
question = "Bạn có biết địa đạo Củ Chi không? Hãy cho tôi một vài thông tin về địa điểm này."
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)
print(ai_msg_1['answer'])

Địa đạo Củ Chi là một mạng lưới đường hầm kết nối nằm ở huyện Củ Chi, Thành phố Hồ Chí Minh, Việt Nam. Các đường hầm được sử dụng bởi quân du kích Việt Cộng như là nơi ẩn náu trong chiến tranh Việt Nam và là nơi đóng quân, bệnh viện và kho vũ khí. Du khách có thể bò qua các đường hầm ngắn hơn hiện đã được bê tông hóa.


In [11]:

second_question = "Hệ thống đường hầm của nó dài bao nhiêu?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Hệ thống đường hầm của nó dài hơn 120 km.


In [12]:
chat_history

[HumanMessage(content='Bạn có biết địa đạo Củ Chi không? Hãy cho tôi một vài thông tin về địa điểm này.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Địa đạo Củ Chi là một mạng lưới đường hầm kết nối nằm ở huyện Củ Chi, Thành phố Hồ Chí Minh, Việt Nam. Các đường hầm được sử dụng bởi quân du kích Việt Cộng như là nơi ẩn náu trong chiến tranh Việt Nam và là nơi đóng quân, bệnh viện và kho vũ khí. Du khách có thể bò qua các đường hầm ngắn hơn hiện đã được bê tông hóa.', additional_kwargs={}, response_metadata={})]

In [57]:
delete_collection()

Successfully deleted collection RAG-Chatbot-Collection-Test


True